In [1]:
from manim import *
from common import nuclear
from importlib import reload
reload(nuclear)

Manim Community v0.17.3

<module 'common.nuclear' from '/home/mic/Documents/code/python/boldNotes/common/nuclear.py'>

In [21]:
%%manim -v WARNING --disable_caching -qm MainScene

config.media_width = "75%"
config.media_embed = True

class MainScene(Scene):
    def construct(self):
        nucleus = nuclear.Nucleus().init_from_nums(12, 12, 0.5, 4, seed=6)
        # Alpha decay
        self.play(*nucleus.create_anims())  # show the nucleus on screen
        daughter1, daughter2, pairs1, pairs2 = nucleus.decay(2, 2, (2, -1), False, True, 1)

        daughter1.shift(2 * RIGHT + DOWN)

        self.play(*map(lambda p: ReplacementTransform(p[0], p[1]), pairs1 + pairs2))
        # self.play(*map(lambda p: Transform(p[0], p[1]), pairs2))
        self.wait(1)
        # Gamma decay
        new_daughter2 = nuclear.Nucleus().init_from_nucleons(daughter2.nucleons, 0.5, 4, True, seed=2)
        new_daughter2.shift(3*LEFT)
        self.play(Transform(daughter2, new_daughter2))